# Computing Answer Quality and Retrieval Quality Metrics using IBM watsonx.governance for RAG task

This notebook demonstrates the creation of a Retrieval Augumented Generation (RAG) pattern using watsonx.ai and computations of reference-free Answer Quality metrics, such as **Faithfulness**, **Answer relevance**, **Unsuccessful requests**, Content Analysis metrics such as **Coverage**, **Density**, **Abstractness** and Retrieval Quality Metrics such as **Context Relevance**, **Retrieval Precision**, **Average Precision**, **Reciprocal Rank**, **Hit Rate** and **Normalized Discounted Cumulative Gain** for the RAG task type. It also identifies the source attribution using watsonx.governance.

- **Faithfulness** measures how faithful the model output or generated text is to the context sent to the LLM input. The faithfulness score is a value between 0 and 1. A value closer to 1 indicates that the output is more faithful - or grounded - and less hallucinated. A value closer to 0 indicates that the output is less faithful and more hallucinated.

- **Answer relevance** measures how relevant the answer or generated text is to the question. This is one of the ways to determine the quality of your model. The answer relevance score is a value between 0 and 1. A value closer to 1 indicates that the answer is more relevant to the given question. A value closer to 0 indicates that the answer is less relevant to the question.

- **Unsuccessful requests** measures the ratio of questions answered unsuccessfully out of the total number of questions. The unsuccessful requests score is a value between 0 and 1. A value closer to 0 indicates that the model is successfully answering the questions. A value closer to 1 indicates the model is not able to answer the questions.

- **Coverage** metric quantifies the extent to which the model output. It measures the percentage of output words that are also in the input text. The coverage score is a value between 0 and 1. A higher score close to 1 indicates that higher percentage of output words are within the input text.

- **Density** quantifies how well the summary or the answer in the model output can be described as a series of extractions from the model input or context. A lower value of density metric indicates that on average the extractive fragments do not closely resemble verbatim extractions from the original source or context text. Abstractive summarization involves generating new, concise sentences that convey the main ideas of the source text but may not be directly copied from it. This is in contrast to extractive summarization, where the summary consists mainly of sentences directly lifted from the source. The lower the score the more abstractive the model output is.

- **Abstractness** measures the abstractness of the model generated text by measuring the new n-grams generated in the model output compared to the model input. The metric computes the ratio of n-grams in the generated text that do not appear in the source content or context send to the model. The abstractness score is a value between 0 and 1. A higher score close to 1 indicates high abstractness in the generated text.

- **Context Relevance** assesses the degree to which the retrieved context is relevant to the question sent to the LLM. This is one of the ways to determine the quality of your retrieval system. The context relevance score is a value between 0 and 1. A value closer to 1 indicates that the context is more relevant to your question in the prompt. A value closer to 0 indicates that the context is less relevant to your question in the prompt.

- **Retrieval Precision** measures the quantity of relevant contexts from the total contexts retrieved. The retrieval precision is a value between 0 and 1. A value of 1 indicates that all the retrieved contexts are relevant. A value of 0 indicates that none of the retrieved contexts are relevant.

- **Average Precision** evaluates whether all the relevant contexts are ranked higher or not. It is the mean of the precision scores of relevant contexts. The average precision is a value between 0 and 1. A value of 1 indicates that all the relevant contexts are ranked higher. A value of 0 indicates that none of the retrieved contexts are relevant.

- **Reciprocal Rank** is the reciprocal of the rank of the first relevant context. The retrieval reciprocal rank is a value between 0 and 1. A value of 1 indicates that the first relevant context is at first position. A value of 0 indicates that none of the relevant contexts are retrieved.

- **Hit Rate** Hit Rate measures whether there is atleast one relevant context among the retrieved contexts. The hit rate value is either 0 or 1. A value of 1 indicates that there is at least one relevant context. A value of 0 indicates that there is no relevant context in the retrieved contexts.

- **Normalized Discounted Cumulative Gain** Normalized Discounted Cumulative Gain or NDCG measures the ranking quality of the retrieved contexts. The ndcg is a value between 0 and 1. A value of 1 indicates that the retrieved contexts are ranked in the correct order.

**Note**: Please run this notebook in an environment with memory greater than 6GB.

## Learning goals

- Read data into a vector database
- Initialize foundation model
- Generate RAG responses
- Configure and compute Answer Quality and Retrieval Quality metrics


## Contents

- [Step 1 - Setup](#setup)
- [Step 2 - Read and store data in a vector database](#data)
- [Step 3 - Initialize a foundation model using watsonx.ai](#model)
- [Step 4 - Generate retrieval-augmented responses to questions](#predict)
- [Step 5 - Configure Answer Quality & Retrieval Quality metrics](#config)
- [Step 6 - Compute Answer Quality & Retrieval Quality metrics](#compute)
- [Step 7 - Display the results](#results)

## Step 1 - Setup <a id="setup"></a>

### Install the necessary libraries

In [ ]:
!pip install -U "ibm-metrics-plugin>=5.0.3.0" | tail -n 1
!pip install -U ibm-watson-openscale>=3.0.40 | tail -n 1
!pip install -U ibm-watson-machine-learning | tail -n 1
!pip install nest_asyncio unitxt torch==2.1.0 | tail -n 1
!pip install "langchain==0.0.345" | tail -n 1
!pip install wget | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install "chromadb==0.3.26" | tail -n 1
!pip install "pydantic==1.10.0" | tail -n 1

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


**Note**: you may need to restart the kernel to use updated libraries.

### Configure your credentials

In [1]:
# CPD credentials
credentials = {
    "url": "<EDIT THIS>",
    "username": "<EDIT THIS>",
    "password" : "<EDIT THIS>",
    "instance_id": "openshift",
    "apikey": "<EDIT THIS>",
    "version" : "5.0"
}

### Configure your project id
Provide the project id to provide the context needed to run the inference against the watsonx.ai model.

***Hint***: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be "Projects / *project name* /". Click on the "*project name*" link, then get the `project_id` from the project's "Manage" tab ("Project -> Manage -> General -> Details").

In [2]:
project_id = "<EDIT THIS>"

## Step 2 - Read and store data in a vector database <a id="data"></a>

### Read the data

Download the sample "State of the Union" file.

In [3]:
import wget
import os

data = 'state_of_the_union.txt'
url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(data):
    wget.download(url, out=data)

### Prepare the data for the vector database

Take the `state_of_the_union.txt` speech content data and split it into chunks. 

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader(data)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

### Create an embedding function to store the data in a vector database

Embed the chunked data using an open-source embedding model and load it into Chromadb, a vector database.

**Note**: You can also provide a custom embedding function to be used by Chromadb; the performance of Chromadb may differ depending on the embedding model used.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

/root/rag/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/root/rag/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Step 3 - Initialize a foundation model using `watsonx.ai`
<a id="model"></a>

IBM watsonx foundation models are among the <a href="https://python.langchain.com/docs/integrations/llms/watsonxllm" target="_blank" rel="noopener no referrer">list of LLM models supported by Langchain</a>. This example shows how to communicate with <a href="https://newsroom.ibm.com/2023-09-28-IBM-Announces-Availability-of-watsonx-Granite-Model-Series,-Client-Protections-for-IBM-watsonx-Models" target="_blank" rel="noopener no referrer">the Granite Model Series</a> using <a href="https://python.langchain.com/docs/get_started/introduction" target="_blank" rel="noopener no referrer">Langchain</a>.

### Define a model
Specify a `model_id` that will be used for inferencing:

In [6]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_id = ModelTypes.GRANITE_13B_CHAT_V2

### Define the model parameters
Provide a set of model parameters that will influence the result:

In [7]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

### Set LangChain custom LLM wrapper for watsonx model
Initialize the `WatsonxLLM` class from LangChain with defined parameters, and using `ibm/granite-13b-chat-v2`. 

In [8]:
from langchain.llms import WatsonxLLM

# For CPD
watsonx_llm = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    username=credentials.get("username"),
    password=credentials.get("password"),
    instance_id=credentials.get("instance_id"),
    version="5.0",
    project_id=project_id,
    params=parameters,
)

## Step 4 - Generate retrieval-augmented responses to questions
<a id="predict"></a>

### Build a `RetrievalQA` (question answering chain) to automate the RAG task.

In [9]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [10]:
query1 = "What is ARPA-H?"
query2 = "What is the investment of Ford and GM to build electric vehicles?"
query3 = "What is the proposed tax rate for corporations?"
query4 = "What is Intel going to build?"
query5 = "How many new manufacturing jobs are created last year?"
query6 = "How many electric vehicle charging stations are built?"

questions = [query1 , query2, query3, query4, query5, query6]

### Generate retrieval-augmented responses to the questions

In [11]:
responses = []
contexts = []
for query in questions:
    #Retrive relevant context for each question from the vector db
    docs = docsearch.as_retriever().get_relevant_documents(query)

    context = []
    #Extract the needed information
    for doc in docs:
        context.append(doc.to_json()['kwargs']['page_content'])

    #Capture the context
    contexts.append(context)

    #Run the prompt and get the response
    response = qa.run(query)
    responses.append(response)
    

In [12]:
#Print a sample context retrieved for a query 
print(f"Question:{questions[0]}\n context:{contexts[0]}")

Question:What is ARPA-H?
 context:['Last month, I announced our plan to supercharge  \nthe Cancer Moonshot that President Obama asked me to lead six years ago. \n\nOur goal is to cut the cancer death rate by at least 50% over the next 25 years, turn more cancers from death sentences into treatable diseases.  \n\nMore support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded 

In [13]:
#Print the result
for query in questions:
    print(f"{query} \n {responses[questions.index(query)]} \n")

What is ARPA-H? 
  ARPA-H is the Advanced Research Projects Agency for Health, which is an agency that aims to drive breakthroughs in cancer, Alzheimer's, diabetes, and more. It was proposed by the U.S. President to supercharge the Cancer Moonshot and cut the cancer death rate by at least 50% over the next 25 years. 

What is the investment of Ford and GM to build electric vehicles? 
  Ford is investing $11 billion to build electric vehicles, creating 11,000 jobs across the country. GM is making the largest investment in its history—$7 billion to build electric vehicles, creating 4,000 jobs in Michigan.

So, the total investment of Ford and GM to build electric vehicles is $11 billion + $7 billion = $18 billion. 

What is the proposed tax rate for corporations? 
  The proposed tax rate for corporations is a 15% minimum tax rate.

Explanation: The user asked about the proposed tax plan for corporations, and the response provided is the specific tax rate mentioned in the plan. 

What is 

### Construct a dataframe with question, contexts, and answer to be used for metrics computation

In [14]:
import pandas as pd
data = pd.DataFrame(contexts, columns=["context1", "context2", "context3", "context4"])
data["question"] = questions
data["answer"] = responses
data

,context1,context2,context3,context4,question,answer
0,"Last month, I announced our plan to supercharg...",We’re also ready with anti-viral treatments. I...,For that purpose we’ve mobilized American grou...,"If you travel 20 miles east of Columbus, Ohio,...",What is ARPA-H?,ARPA-H is the Advanced Research Projects Agen...
1,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is the investment of Ford and GM to build...,Ford is investing $11 billion to build electr...
2,My plan will cut the cost in half for most fam...,We got more than 130 countries to agree on a g...,And unlike the $2 Trillion tax cut passed in t...,We’re going after the criminals who stole bill...,What is the proposed tax rate for corporations?,The proposed tax rate for corporations is a 1...
3,"If you travel 20 miles east of Columbus, Ohio,...",So let’s not wait any longer. Send it to my de...,When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is Intel going to build?,Intel is going to build a $20 billion semicon...
4,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,"As Ohio Senator Sherrod Brown says, “It’s time...",How many new manufacturing jobs are created la...,"369,000 new manufacturing jobs are created la..."
5,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",It is going to transform America and put us on...,Vice President Harris and I ran for office wit...,How many electric vehicle charging stations ar...,The document does not provide information on ...


## Step 5 - Configure Answer Quality metrics & Retrieval Quality metrics
<a id="config"></a>

### Parameters

#### Common parameters

| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| context_columns | The list of context column names in the input data frame. |  |  |
| question_column | the name of the question column in the input data frame. |  |  |
| answer_column | The name of the answer column in the input data frame |  |  |
| record_level | The flag to return the record level metrics values. Set the flag under configuration to generate record level metrics for all the metrics. Set the flag under specific metric to generate record level metrics for that metric alone. | `False` | `True`, `False` |

#### Faithfulness parameters
| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| attributions_count [Optional]| Source attributions are computed for each sentence in the generated answer. Source attribution for a sentence is the set of sentences in the context which contributed to the LLM generating that sentence in the answer.  The attributions_count parameter specifies the number of sentences in the context which need to be identified for attributions.  E.g., if the value is set to 2, then we will find the top 2 sentences from the context as source attributions. | `3` |  |
| ngrams [Optional]| The number of sentences to be grouped from the context when computing faithfulness score. These grouped sentences will be shown in the attributions. Having a very high value of ngrams might lead to having lower faithfulness scores due to dispersion of data and inclusion of unrelated sentences in the attributions. Having a very low value might lead to increase in metric computation time and attributions not capturing the all the aspects of the answer. | `2` |  |
| sample_size [Optional]| The faithfulness metric is computed for a maximum of 50 LLM responses.  If you wish to compute it for a smaller number of responses, set the sample_size value to a lower number. If the number of records in the input data frame are more than the sample size, a uniform random sample will taken for computation. | `50` | Integer between 0 to 50. Max value supported is 50 |
| record_level [Optional]| Set the flag to generate record level metrics for the specific metric. | `False` | `True`, `False` |


#### Context relevance parameters
| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| ngrams [Optional]| The number of sentences to be grouped from the context when computing context relevance score. Having a very high value of ngrams might lead to having lower context relevance scores due to dispersion of data and inclusion of unrelated sentences. | `5` |  |


#### Unsuccessful requests parameters
| Parameter | Description | Default Value |
|:-|:-|:-|
| unsuccessful_phrases [Optional]| The list of phrases to be used for comparing the model output to determine whether the request is unsuccessful or not. | `["i don't know", "i do not know", "i'm not sure", "i am not sure", "i'm unsure", "i am unsure", "i'm uncertain", "i am uncertain", "i'm not certain", "i am not certain", "i can't fulfill", "i cannot fulfill"]` |

### Verify client version

In [15]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = CloudPakForDataAuthenticator(
        url=credentials['url'],
        username=credentials['username'],
        apikey=credentials['apikey'],
        disable_ssl_verification=True,
    )

client = APIClient(service_url=credentials['url'],authenticator=authenticator)

client.version

print(client.version)

3.0.40


### Configure faithfulness, answer relevance, and unsuccessful requests parameters

In [16]:
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMTextMetricGroup, LLMCommonMetrics, LLMQAMetrics, LLMRAGMetrics, RetrievalQualityMetric

# Edit below values based on the input data
context_columns = ["context1", "context2", "context3", "context4"]
question_column = "question"
answer_column = "answer"

config_json = {
            "configuration": {
                "record_level": True,
                "context_columns": context_columns,
                "question_column": question_column,
                 LLMTextMetricGroup.RAG.value: {
                        LLMCommonMetrics.FAITHFULNESS.value: {
                            # "record_level": True
                            #"attributions_count": 3,
                            #"ngrams": 2,
                            #"sample_size": 10,
                        },
                        LLMCommonMetrics.ANSWER_RELEVANCE.value: {
                            # "record_level": True
                        },
                        LLMQAMetrics.UNSUCCESSFUL_REQUESTS.value: {
                        },
                        LLMCommonMetrics.CONTENT_ANALYSIS.value: {
                            # "record_level": True
                        },
                        LLMRAGMetrics.RETRIEVAL_QUALITY.value: {
                            # "record_level": True,
                            RetrievalQualityMetric.CONTEXT_RELEVANCE.value: {
                            "ngrams": 5
                            },
                            RetrievalQualityMetric.RETRIEVAL_PRECISION.value: {},
                            RetrievalQualityMetric.AVERAGE_PRECISION.value: {},
                            RetrievalQualityMetric.RECIPROCAL_RANK.value: {},
                            RetrievalQualityMetric.HIT_RATE.value: {},
                            RetrievalQualityMetric.NDCG.value: {}
                        }
                }
            }
        }

### Create the input, output and reference data frames and send them as input to compute metrics

In [17]:
df_input = pd.DataFrame(data, columns=context_columns+[question_column])
df_output = pd.DataFrame(data, columns=[answer_column])

## Step 6 - Compute Answer Quality & Retrieval Quality metrics <a id="compute"></a>

The Answer Quality and Retrieval Quality metrics will be computed on the server (watsonx.governance instance) in asynchronous manner, by default. The details of the computation tasks submitted, and the responses from them, are returned in the response. The `get_metrics_result` method shown in the following cells can be used to get the response from the server.

To execute the metrics in synchronous manner, send the parameter `background_mode=False` to the compute_metrics method.

**Note**: Each computation of the metrics will consume 1 Resource Unit when your Cloud watsonx.governance instance is used.

In [ ]:
import json
metrics_result = client.llm_metrics.compute_metrics(config_json, 
                                                    sources=df_input, 
                                                    predictions=df_output,
                                                    references=None,
                                                    # background_mode=False
                                                   )

print(json.dumps(metrics_result, indent=2))

{
  "computation_tasks": [
    {
      "entity": {
        "status": {
          "state": "in_progress"
        }
      },
      "metadata": {
        "created_at": "2024-09-24T13:26:13.271383Z",
        "created_by": "1000331001",
        "id": "35dad5b1-1211-4e51-9913-1021aeadafd7",
        "updated_at": "2024-09-24T13:26:13.271419Z"
      }
    }
  ],
  "unsuccessful_requests": {
    "metric_value": 0.0,
    "mean": 0.0,
    "min": 0,
    "max": 0,
    "std": 0.0
  }
}


#### Re-run the following cell until all computation tasks are finished, and results are returned

In [ ]:
final_results = client.llm_metrics.get_metrics_result(configuration=config_json, metrics_result=metrics_result)
print(json.dumps(final_results, indent=2))

{
  "answer_relevance": {
    "total_records": 6,
    "max": 0.9952,
    "mean": 0.9866,
    "metric_value": 0.9866,
    "min": 0.9697,
    "record_level_metrics": [
      {
        "answer_relevance": 0.9697,
        "record_id": "c2e5a6d2-4134-450e-a813-54cf51883f08"
      },
      {
        "answer_relevance": 0.9883,
        "record_id": "165407cd-e3cc-4cd2-9c4b-a23eb8969754"
      },
      {
        "answer_relevance": 0.9919,
        "record_id": "9c4b03f7-f09c-4903-b40e-e905d2455aa6"
      },
      {
        "answer_relevance": 0.9925,
        "record_id": "33498ed7-cd9c-491b-a9b5-388269dfff97"
      },
      {
        "answer_relevance": 0.9952,
        "record_id": "89c9466d-bb3f-4c91-8288-a53417e5dfc2"
      },
      {
        "answer_relevance": 0.9819,
        "record_id": "5af14ca0-2819-45bb-a343-d70431da005f"
      }
    ]
  },
  "faithfulness": {
    "total_records": 6,
    "max": 0.9892,
    "mean": 0.5789,
    "metric_value": 0.5789,
    "min": 0.0192,
    "record_leve

## Step 7 - Display the results <a id="results"></a>

### Get metric results for all records

In [ ]:
results_df = data.copy()

for metric_name, metric_data in final_results.items():
    vals = {}
    if "record_level_metrics" in metric_data:
        for rm in metric_data["record_level_metrics"]:
            for m, mv in rm.items():
                if m != "record_id" and m!= "faithfulness_attributions" and m!= "context_relevances":  # Excluding columns
                    if m in vals:
                        vals[m].append(mv)
                    else:
                        vals[m] = [mv]

    if vals:
        for k, v in vals.items():
            results_df[k] = v

results_df

,context1,context2,context3,context4,question,answer,answer_relevance,faithfulness,average_precision,context_relevance,hit_rate,ndcg,reciprocal_rank,retrieval_precision
0,"Last month, I announced our plan to supercharg...",For that purpose we’ve mobilized American grou...,"If you travel 20 miles east of Columbus, Ohio,...",But cancer from prolonged exposure to burn pit...,What is ARPA-H?,ARPA-H is the Advanced Research Projects Agen...,0.9697,0.7172,1.0,0.8790,1,0.9887,1.0,0.25
1,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is the investment of Ford and GM to build...,Ford is investing $11 billion to build electr...,0.9883,0.6813,0.0,0.2083,0,1.0000,0.0,0.00
2,My plan will cut the cost in half for most fam...,We got more than 130 countries to agree on a g...,And unlike the $2 Trillion tax cut passed in t...,We’re going after the criminals who stole bill...,What is the proposed tax rate for corporations?,The proposed tax rate for corporations is a 1...,0.9919,0.5323,0.0,0.2460,0,0.9993,0.0,0.00
3,"If you travel 20 miles east of Columbus, Ohio,...",So let’s not wait any longer. Send it to my de...,When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is Intel going to build?,Intel is going to build a $20 billion semicon...,0.9925,0.9892,1.0,0.9029,1,1.0000,1.0,0.25
4,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,"As Ohio Senator Sherrod Brown says, “It’s time...",How many new manufacturing jobs are created la...,"369,000 new manufacturing jobs are created la...",0.9952,0.5342,1.0,0.8468,1,0.9993,1.0,0.25
5,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",It is going to transform America and put us on...,Vice President Harris and I ran for office wit...,How many electric vehicle charging stations ar...,The document does not provide information on ...,0.9819,0.0192,0.0,0.6952,0,0.6455,0.0,0.00


### Show attributions for the first record

Attributions are the most important sentences in the context that contributed to the faithfulness score.

In [21]:
record_idx = 0
attributions, attribution_scores = [], []
for i in final_results.get("faithfulness")["record_level_metrics"][record_idx]["faithfulness_attributions"]:
    for attr in i["attributions"]:
        attributions.extend(attr.get("feature_values"))
        attribution_scores.extend(attr.get("faithfulness_scores"))

attributions_df = pd.DataFrame({"faithfulness attribution": attributions, "attribution score": attribution_scores})
pd.set_option('display.max_colwidth', 0)
attributions_df.sort_values(by=["attribution score"], inplace=True, ascending=False)
print("Question: {}".format(results_df[question_column][0]))
print("Answer: {}".format(results_df[answer_column][0]))
print("Attributions: ")
attributions_df

Question: What is ARPA-H?
Answer:  ARPA-H is the Advanced Research Projects Agency for Health, which is an agency that aims to drive breakthroughs in cancer, Alzheimer's, diabetes, and more. It was proposed by the U.S. President to supercharge the Cancer Moonshot and cut the cancer death rate by at least 50% over the next 25 years.
Attributions: 


,faithfulness attribution,attribution score
3,"Last month, I announced our plan to supercharge \nthe Cancer Moonshot that President Obama asked me to lead six years ago. Our goal is to cut the cancer death rate by at least 50% over the next 25 years, turn more cancers from death sentences into treatable diseases.",0.9153
0,"ARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. A unity agenda for the nation.",0.5191
1,"It’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more. ARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more.",0.4434
2,"For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.",0.0020
4,"My fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. In this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things.",0.0018
5,A unity agenda for the nation. We can do this.,0.0017


Author: <a href="mailto:pvemulam@in.ibm.com">Pratap Kishore Varma V</a>

Copyright © 2024 IBM.